In [1]:
%run ../../utils/commonImports.py
%run ../../utils/tradingImports.py
%matplotlib inline

from base import supres, BaseStrategy

# Load Data

In [2]:
# dropbox_dir = 'D:\\Dropbox\\My work\\krypl-project'
dropbox_dir = '/Users/david.vesely/Dropbox/My work/krypl-project'

def read_data(pair):
    db = os.path.join(dropbox_dir,'sqlite', 'ploniex-chart-data', f'{pair}.db')
    data = load_trading_data(db, 'chart_data', from_date='2015-01-01', period='30min')
    data['ohlc4'] = (data['open'] + data['close'] + data['high'] + data['low']) / 4
    return data

In [3]:
supports_for_data_dict_test = load_model('supports-for-data-test.pkl')
supports_for_data_dict_train = load_model('supports-for-data.pkl')
supports_for_data_dict = {**supports_for_data_dict_train, **supports_for_data_dict_test}

In [4]:
features = ['support']

In [5]:
train_pairs = pd.read_csv('pairs_train.tsv', header=None)[0].tolist()
test_pairs = pd.read_csv('pairs_test.tsv', header=None)[0].tolist()
all_pairs = pd.read_csv('pairs.tsv', header=None)[0].tolist()
data_dict = {pair: read_data(pair) for pair in all_pairs}

# Parallelize

In [6]:
from multiprocessing import Pool
import time
from tqdm import *

def run_parallel(f, args, n_process=10):
    results = []
    with Pool(processes=n_process) as p:
        all_args = list(args)
        with tqdm_notebook(enumerate(p.imap_unordered(f, all_args)), leave=False, total=len(all_args)) as pbar:
            for i, x in pbar:
                results.append(x)
                pbar.update()
                
        return results

In [7]:
def dict_from_list(l):
    d = {}
    for row in l:
        d_inner = d
        for k in row[:-2]:
            if k not in d_inner.keys():
                d_inner[k] = {}
            d_inner = d_inner[k]
        d_inner[row[-2]] = row[-1]
    return d

# Multicoin strategy

In [8]:
class MulticoinStrategy():
    def __init__(self, data_dict, pairs, willing_loss, target_profit, target_return):
        self.data_dict = data_dict
        self.pairs = pairs
        self.willing_loss = willing_loss
        self.target_profit = target_profit
        self.target_return = target_return
        self.strategies = {}
        self.wallet = {'btc': 100}
    
    
    def trade_pair(self, pair):
        contract_pair = ContractPair.new(*[x.lower() for x in pair.split('_')])
        data = self.data_dict[pair]
        dm = OhlcDataManager(data[['open', 'low', 'high', 'close']], data[features])
        wallet = deepcopy(self.wallet)
        wallet[contract_pair['priceContract']] = wallet[contract_pair['priceContract']] / len(self.pairs)
        exchange = BackTestExchange(dm, wallet, 0.0025)
        strategy = BaseStrategy(exchange, 
                                dm, 
                                contract_pair, 
                                willing_loss=self.willing_loss, 
                                target_profit=self.target_profit, 
                                target_return=self.target_return)
        strategy.trade()
        return pair, strategy
    
    def trade(self):
        results = run_parallel(self.trade_pair, self.pairs)
        self.strategies = dict_from_list(results)
    
    
    def all_transactions(self):
        transactions = []
        for strategy in self.strategies.values():
            transactions += strategy.exchange.transactions
        return transactions
    
    def transactions(self, contract_name=None):
        if contract_name is None:
            return self.all_transactions()
        
        return self.strategies[contract_name].exchange.transactions
    
    def stats(self, contract_name):
        return Statistics(contract_name, self.wallet[contract_name])\
            .evaluate(self.all_transactions())

# Setup

In [9]:
window_size = 12
target_profit = 0.01
willing_loss = 0.06
target_return = -0.02

for pair in all_pairs:
    data = data_dict[pair]
    data['support'] = supports_for_data_dict[pair][window_size]

# Train Results

In [10]:
multi_strategy = MulticoinStrategy(data_dict, train_pairs, willing_loss, target_profit, target_return)
multi_strategy.trade()
multi_strategy.stats('btc').report()

,btc
startAmount,100.0000
numberOfTrades,"7,530.0000"
totalProfit,474.4518
avgProfit,0.0630
winPercentage,95.2324
avgWinTrade,0.0987
avgLossTrade,-0.6491
profitFactor,3.0359
maxDrawdown,2.9596
avgTimeToClose,6.5676


In [19]:
transactions = multi_strategy.all_transactions()

for t in transactions:
    pair = t['pair']
    price_c = pair['priceContract'].upper()
    trade_c = pair['tradeContract'].upper()
    pair = f'{price_c}_{trade_c}'
    data = data_dict[pair]
    
    t['timestamp'] = data['timestamp'].iloc[t['timestamp']]
    break

In [20]:
transactions[0]

{'type': 'BUY',
 'pair': {'priceContract': 'btc', 'tradeContract': 'ardr'},
 'timestamp': 1477004400,
 'amount': 141987.82961460444,
 'price': 2.38e-05,
 'fee': 0.008448275862068965}

In [17]:
data\

,timestamp,period,open,high,low,close,volume,quoteVolume,weightedAverage,date,ohlc4,support
0,1476666000,30min,0.0500,0.0500,0.0500,0.0500,0.0001,0.0020,0.0500,2016-10-17 01:00:00,0.0500,nan
1,1476667800,30min,0.0000,0.0001,0.0000,0.0000,63.0028,"1,793,555.4010",0.0000,2016-10-17 01:30:00,0.0000,nan
2,1476669600,30min,0.0000,0.0001,0.0000,0.0000,162.6505,"3,679,260.0040",0.0000,2016-10-17 02:00:00,0.0000,nan
3,1476671400,30min,0.0000,0.0000,0.0000,0.0000,225.4817,"5,059,261.4318",0.0000,2016-10-17 02:30:00,0.0000,nan
4,1476673200,30min,0.0000,0.0000,0.0000,0.0000,195.4411,"5,099,093.0827",0.0000,2016-10-17 03:00:00,0.0000,nan
5,1476675000,30min,0.0000,0.0000,0.0000,0.0000,67.7008,"1,774,996.4818",0.0000,2016-10-17 03:30:00,0.0000,nan
6,1476676800,30min,0.0000,0.0000,0.0000,0.0000,29.8129,"766,691.7097",0.0000,2016-10-17 04:00:00,0.0000,nan
7,1476678600,30min,0.0000,0.0000,0.0000,0.0000,57.1985,"1,586,788.4388",0.0000,2016-10-17 04:30:00,0.0000,nan
8,1476680400,30min,0.0000,0.0000,0.0000,0.0000,60.0141,"1,807,989.6187",0.0000,2016-10-17 05:00:00,0.0000,nan
9,1476682200,30min,0.0000,0.0000,0.0000,0.0000,29.8723,"917,245.5373",0.0000,2016-10-17 05:30:00,0.0000,nan


# Test Results

In [11]:
multi_strategy = MulticoinStrategy(data_dict, test_pairs, willing_loss, target_profit, target_return)
multi_strategy.trade()
multi_strategy.stats('btc').report()

,btc
startAmount,100.0000
numberOfTrades,"7,303.0000"
totalProfit,554.1265
avgProfit,0.0759
winPercentage,95.4676
avgWinTrade,0.1177
avgLossTrade,-0.8052
profitFactor,3.0791
maxDrawdown,4.7714
avgTimeToClose,7.1464
